In [1]:
import zipfile, struct, warnings, requests, struct
from io import StringIO, BytesIO

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import astropy, skimage
from astropy.io.votable import parse_single_table
from astroquery.utils.tap.core import TapPlus
import pvl

from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode()

from IPython.display import Image, display

%matplotlib inline
# warnings.simplefilter('ignore', category=astropy.io.votable.exceptions.VOTableSpecWarning)
# astropy.io.votable.tree
# warnings.simplefilter('ignore', category=skimage.util.dtype.warn)
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
tap_url = 'https://archives.esac.esa.int/psa/epn-tap/tap/'
psa = TapPlus(url=tap_url)